## Deribit API Request  - For BTC and ETH Options Data
#### ALL Options data 

### Specfic Settlement Period ( Day, Week, Month) 

In [1]:
import pandas as pd
import json
import requests
import re
from datetime import datetime
from tqdm import tqdm
import concurrent.futures

# Functions
def get_option_name_and_settlement(coin):
    """
    :param coin: crypto-currency coin name ('BTC', 'ETH')
    :return: 2 lists:
                        1. list of traded options for the selected coin;
                        2. list of settlement period for the selected coin.
    """
    r = requests.get("https://test.deribit.com/api/v2/public/get_instruments?currency=" + coin + "&kind=option")
    result = json.loads(r.text)
    # Get option name
    name = pd.json_normalize(result['result'])['instrument_name']
    name = list(name)

    # Get option settlement period
    settlement_period = pd.json_normalize(result['result'])['settlement_period']
    settlement_period = list(settlement_period)

    return name, settlement_period

def fetch_option_data(option_name):
    """Fetch the option data for a given option name."""
    r = requests.get(f'https://test.deribit.com/api/v2/public/get_order_book?instrument_name={option_name}')
    result = json.loads(r.text)
    return pd.json_normalize(result['result'])


def extract_details(instrument_name, coin):
    """
    Extract expiration date, strike price, and option type (call or put) from instrument name.
    Adjusts to include selected coin (e.g., 'BTC' or 'ETH').
    """
    match = re.match(fr"{coin}-(\d+[A-Z]{{3}}\d+)-(\d+)-([CP])", instrument_name)
    if match:
        expiration_date = match.group(1)
        strike_price = match.group(2)
        option_type = 'Call' if match.group(3) == 'C' else 'Put'
        return expiration_date, strike_price, option_type
    return None, None, None



def get_option_data(coin, settlement_per):
    """
    :param coin: crypto-currency coin name ('BTC', 'ETH')
    :return: pandas DataFrame with all option data for a given coin, filtered for options with specified settlement.
    """
    # Get option name and settlement
    coin_name, settlement_period = get_option_name_and_settlement(coin)

    # Filter options that have the specified settlement period
    coin_name_filtered = [coin_name[i] for i in range(len(coin_name)) if settlement_period[i] == settlement_per]

    # Initialize progress bar
    pbar = tqdm(total=len(coin_name_filtered))

    # Fetch data concurrently using ThreadPoolExecutor
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_option = {executor.submit(fetch_option_data, name): name for name in coin_name_filtered}
        coin_df = []
        
        for future in concurrent.futures.as_completed(future_to_option):
            try:
                data = future.result()
                data['settlement_period'] = settlement_per
                coin_df.append(data)
            except Exception as exc:
                print(f'Error fetching data: {exc}')
            pbar.update(1)

    # Finalize DataFrame
    if len(coin_df) > 0:
        coin_df = pd.concat(coin_df)

    # Remove unnecessary columns
    columns = ['state', 'estimated_delivery_price']
    if not coin_df.empty:
        coin_df.drop(columns, inplace=True, axis=1)

    #coin_df['Expiration Date'], coin_df['Strike Price'] = zip(*coin_df['instrument_name'].apply(lambda x: extract_details(x, coin)))

    coin_df['Expiration Date'], coin_df['Strike Price'], coin_df['Option Type'] = zip(*coin_df['instrument_name'].apply(lambda x: extract_details(x, coin)))
    
    today = datetime.today()
    coin_df['Time to Expiration'] = coin_df['Expiration Date'].apply(lambda x: (datetime.strptime(x, '%d%b%y') - today).days / 365 if x else None)

    coin_df.to_csv('data/data.csv', index=False)
 
    pbar.close()

    return coin_df


In [2]:
data = get_option_data('BTC','day')
data=data[["instrument_name", "Option Type", 
           'mark_price', 'underlying_price', 'mark_iv', 
           'greeks.vega', 'Expiration Date', 'Strike Price', 'Time to Expiration']]

data['Strike Price'] = pd.to_numeric(data['Strike Price'], errors='coerce').astype('float64')

 99%|█████████▉| 125/126 [00:16<00:00, 17.26it/s]C:\Users\Ethan Vaz Falcao\AppData\Local\Temp\ipykernel_19096\1889503572.py:81: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  coin_df = pd.concat(coin_df)
100%|██████████| 126/126 [00:16<00:00,  7.69it/s]


In [3]:
data.isnull().sum()

instrument_name       0
Option Type           0
mark_price            0
underlying_price      0
mark_iv               0
greeks.vega           0
Expiration Date       0
Strike Price          0
Time to Expiration    0
dtype: int64

In [6]:
data

,instrument_name,Option Type,mark_price,underlying_price,mark_iv,greeks.vega,Expiration Date,Strike Price,Time to Expiration
0,BTC-31OCT24-66500-P,Put,0.0001,72459.8528,92.94,0.82361,31OCT24,66500.0,0.000000
0,BTC-31OCT24-69000-C,Call,0.0481,72459.8528,59.72,2.32569,31OCT24,69000.0,0.000000
0,BTC-31OCT24-69000-P,Put,0.0001,72459.8528,59.72,0.89009,31OCT24,69000.0,0.000000
0,BTC-31OCT24-68500-C,Call,0.0547,72459.8528,59.72,0.25512,31OCT24,68500.0,0.000000
0,BTC-31OCT24-66500-C,Call,0.0823,72459.8528,92.94,0.34245,31OCT24,66500.0,0.000000
...,...,...,...,...,...,...,...,...,...
0,BTC-24JAN25-95000-P,Put,0.3343,72524.8180,54.73,94.89448,24JAN25,95000.0,0.232877
0,BTC-24JAN25-105000-P,Put,0.4596,72524.8180,54.73,63.31825,24JAN25,105000.0,0.232877
0,BTC-24JAN25-105000-C,Call,0.0118,72524.8180,54.73,63.31825,24JAN25,105000.0,0.232877
0,BTC-24JAN25-110000-C,Call,0.0083,72524.8180,54.99,50.56475,24JAN25,110000.0,0.232877


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 126 entries, 0 to 0
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   instrument_name     126 non-null    object 
 1   Option Type         126 non-null    object 
 2   mark_price          126 non-null    float64
 3   underlying_price    126 non-null    float64
 4   mark_iv             126 non-null    float64
 5   greeks.vega         126 non-null    float64
 6   Expiration Date     126 non-null    object 
 7   Strike Price        126 non-null    float64
 8   Time to Expiration  126 non-null    float64
dtypes: float64(6), object(3)
memory usage: 9.8+ KB


In [5]:
data.to_csv('data/data.csv', index=False)